In [1]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup

Будем скрейпить одну таблицу с Википедии

In [2]:
url_wiki = requests.get('https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D1%8F%D0%BD%D0%BD%D1%8B%D1%85_%D0%BF%D0%B0%D0%BC%D1%8F%D1%82%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2_%D0%B0%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D1%83%D1%80%D1%8B_%D0%92%D0%BE%D0%BB%D0%BE%D0%B3%D0%B4%D1%8B').text

In [3]:
soup = BeautifulSoup(url_wiki, 'html.parser')

Находим нужную таблицу на странице

In [4]:
tables = soup.find_all('table', { "class" : "standard" })[0]
rows = tables.find_all('tr')

In [32]:
data = []
colors = []
color = None
for row in rows:
    cols = row.find_all('td')
    for i in cols:
        if i.get('bgcolor') != None:
            color = i.get('bgcolor')
    cols = [ele.text.strip() for ele in cols]
    cols.append(color)
    data.append(cols)

In [33]:
for i in data:
    if len(i) < 8:
        data.remove(i)

In [38]:
houses_df = pd.DataFrame(data)

In [40]:
houses_df.columns = ['id', 'more_id', 'type', 'year', 'address', 'comment', 'category', 'num1', 'num2', 'status_color']

In [41]:
houses_df.head()

,id,more_id,type,year,address,comment,category,num1,num2,status_color
0,1,,Дом жилой,XIX век,"набережная VI Армии, 41",,региональная,Объект культурного наследия:№ 3500068000,Объект культурного наследия:№ 3500068000,Black
1,2,,Дом жилой,первая половина XIX века,"набережная VI Армии, 51",жилой дом,региональная,Объект культурного наследия:№ 3500000840,Объект культурного наследия:№ 3500000840,DarkBlue
2,3,,Дом жилой (с дворовым пристроем),первая половина XIX века,"набережная VI Армии, 81",,региональная,Объект культурного наследия:№ 3500070001,Объект культурного наследия:№ 3500070001,Black
3,4,,Дом жилой,третья четверть XIX века,"набережная VI Армии, 91",,региональная,Объект культурного наследия:№ 3500000842,Объект культурного наследия:№ 3500000842,Black
4,5,,Дом жилой,третья четверть XIX века,"набережная VI Армии, 93",,региональная,Объект культурного наследия:№ 3500000844,Объект культурного наследия:№ 3500000844,Black


В колонке цвета зашифровано состояние дома. Нужно её преобразовать

In [42]:
def get_state(row):
    if row == 'Black':
        return 'Утрачен полностью или значительно разрушен'
    elif row == 'Chocolate':
        return 'Хорошее (удовлетворительное)'
    elif row == 'DarkBlue':
        return 'Отреставрирован со значительными нарушениями первоначального облика'
    elif row == 'DarkRed':
        return 'Аварийное'
    elif row == 'Lime':
        return 'Отреставрирован'

In [43]:
houses_df['status'] = houses_df['status_color'].apply(get_state)

Убираем ненужные колонки и сохраняем датасет

In [45]:
houses_df.drop(['id', 'more_id', 'num2'], axis=1, inplace=True)

In [46]:
houses_df.head()

,type,year,address,comment,category,num1,status_color,status
0,Дом жилой,XIX век,"набережная VI Армии, 41",,региональная,Объект культурного наследия:№ 3500068000,Black,Утрачен полностью или значительно разрушен
1,Дом жилой,первая половина XIX века,"набережная VI Армии, 51",жилой дом,региональная,Объект культурного наследия:№ 3500000840,DarkBlue,Отреставрирован со значительными нарушениями п...
2,Дом жилой (с дворовым пристроем),первая половина XIX века,"набережная VI Армии, 81",,региональная,Объект культурного наследия:№ 3500070001,Black,Утрачен полностью или значительно разрушен
3,Дом жилой,третья четверть XIX века,"набережная VI Армии, 91",,региональная,Объект культурного наследия:№ 3500000842,Black,Утрачен полностью или значительно разрушен
4,Дом жилой,третья четверть XIX века,"набережная VI Армии, 93",,региональная,Объект культурного наследия:№ 3500000844,Black,Утрачен полностью или значительно разрушен


In [49]:
houses_df.to_csv('houses.csv')